Encoder Network

In [2]:
%cd /content/drive/MyDrive/Food Pantry Project Information

/content/drive/MyDrive/Food Pantry Project Information


In [10]:
import numpy as np
import pandas as pd

def normalize(data):
    data_mean = data.mean(axis=0)
    data_std = data.std(axis=0)
    return (data - data_mean) / data_std

csv_path = "LakeviewFp.xlsx"
df = pd.read_excel(csv_path)

feature_keys = [
    "Week",
    "Month",
    "Clients",
    "Class",
    "Temperature",
    "Rain",
    "Covid",
    "CPI",
    "ER",
    "School",
    "PI"
]

u=4
start=u

#Features
selected_features = [feature_keys[i] for i in [0, 2, 5, 6, 7, 8, 9]]
features = df[selected_features].values
l=len(features)
print(l)
x=[]
for j in range(l-u):
    y=[features[j+u][1]]
    for k in range(u):
        y=np.append(y, features[j+k])
    x.append(y.tolist())
features=np.array(x)
print(len(features))

np.random.seed(1)
a=np.arange(len(features))
np.random.shuffle(a) 

features_new=[]
y_new=[]
f_new=[]
for i in a:
    features_new.append(features[i][1:])
    y_new.append(features[i][0])
    f_new.append(features[i])

#inference data split
split=1
train_split=round(split*len(features))
print(train_split)
y_test=y_new[train_split:]
x_test=features_new[train_split:]
features=f_new[0:train_split-1]
features=np.array(features)
print(features)
M=features.mean(axis=0)
S=features.std(axis=0)
features = normalize(features)


340
336
336
[[ 45.          3.         45.        ... 103.5870845   4.2
    1.       ]
 [ 42.          3.         56.        ... 102.8622419   4.6
    1.       ]
 [ 45.          2.         18.        ... 108.4200711   2.4
    1.       ]
 ...
 [ 41.          4.         39.        ... 109.8967585   3.3
    1.       ]
 [ 24.          4.         28.        ... 103.3491269   4.6
    0.       ]
 [ 18.          1.         30.        ... 109.3174751   7.4
    0.       ]]


In [11]:
import keras
from keras import layers
from keras import backend as K
import numpy as np

original_dim = 29
intermediate_dim = 10
latent_dim = 2

inputs = keras.Input(shape=(original_dim,))
h = layers.Dense(intermediate_dim, activation='relu')(inputs)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])

# Create encoder
encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 29)]         0           []                               
                                                                                                  
 dense_8 (Dense)                (None, 10)           300         ['input_3[0][0]']                
                                                                                                  
 dense_9 (Dense)                (None, 2)            22          ['dense_8[0][0]']                
                                                                                                  
 dense_10 (Dense)               (None, 2)            22          ['dense_8[0][0]']                
                                                                                            

Decoder Network

In [12]:
# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='linear')(latent_inputs)
outputs = layers.Dense(original_dim, activation='linear')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')

decoder.summary()

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 z_sampling (InputLayer)     [(None, 2)]               0         
                                                                 
 dense_11 (Dense)            (None, 10)                30        
                                                                 
 dense_12 (Dense)            (None, 29)                319       
                                                                 
Total params: 349
Trainable params: 349
Non-trainable params: 0
_________________________________________________________________


In [13]:
reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

In [14]:
np.random.seed(1)
vae.fit(features, features,
        epochs=500,
       batch_size=10,)

Epoch 1/500
34/34 [==============================] - 1s 2ms/step - loss: 1.6279
Epoch 2/500
34/34 [==============================] - 0s 2ms/step - loss: -3.6550
Epoch 3/500
34/34 [==============================] - 0s 2ms/step - loss: -6.7067
Epoch 4/500
34/34 [==============================] - 0s 2ms/step - loss: -12.8166
Epoch 5/500
34/34 [==============================] - 0s 2ms/step - loss: -17.1213
Epoch 6/500
34/34 [==============================] - 0s 2ms/step - loss: -18.5105
Epoch 7/500
34/34 [==============================] - 0s 2ms/step - loss: -21.3154
Epoch 8/500
34/34 [==============================] - 0s 2ms/step - loss: -22.6690
Epoch 9/500
34/34 [==============================] - 0s 2ms/step - loss: -25.1395
Epoch 10/500
34/34 [==============================] - 0s 2ms/step - loss: -24.9896
Epoch 11/500
34/34 [==============================] - 0s 2ms/step - loss: -26.9341
Epoch 12/500
34/34 [==============================] - 0s 2ms/step - loss: -28.2139
Epoch 13/500
34/3

In [ ]:
def denormalize(data, data_mean, data_std):
    return  data*data_std+data_mean

In [ ]:
i=np.random.randint(0,302)
a=np.array([features[i]])
print(denormalize(a, M, S))
b=np.array(encoder.predict(a))
print(b[2])
c=decoder.predict(b[1])
print(denormalize(c, M, S))

error=0
for i in range(300):
  a=np.array([features[i]])
  client=denormalize(a, M, S)[0][3]
  b=np.array(encoder.predict(a))
  c=decoder.predict(b[2])
  decoded_client=denormalize(c, M, S)[0][3]
  error=error+abs(client-decoded_client)
error=error/300
print(error)

[[47. 28. 19. 58. 58.]]
[[-0.49626002]]
[[41.62168909 42.44619989 42.66083663 41.80325134 38.39936176]]
10.04298894586175


In [ ]:
f=[]
for i in range(500):
    i=np.random.randint(0,train_split-1)
    a=np.array([features[i]])
    b=np.array(encoder.predict(a))
    c=decoder.predict(b[2])
    c=denormalize(c, M, S)
    f.append(c[0])
for j in range(len(features)):
  c=denormalize(features[i], M, S)
  f.append(c)
f=np.array(f)

In [ ]:
print(f)

In [ ]:
print(f[0][1:])

In [ ]:

print(len(f))
l=len(f)
print(l)
#Target
y=[]
#feature
x=[]
for j in range(l):
    y.append(f[j][0])
    x.append(f[j][1:])
print(y) 

In [ ]:
print(y[0])
print(x[0])

In [ ]:
print(f[1])

In [ ]:
np.random.seed(1)
a=np.arange(len(y))
np.random.shuffle(a) 
features_new=[]
y_new=[]
for i in a:
    features_new.append(x[i])
    y_new.append(y[i])
features=np.array(features_new)
y=y_new
    
#Training data and Validation data
split=0.8
train_split=round(len(y)*0.8)
x_train=x[0:train_split-1]
x_val=x[train_split:]
y_train=y[0:train_split-1]
y_val=y[train_split:]
x_train=np.array(x_train)
x_val=np.array(x_val)
y_train=np.array(y_train)
y_val=np.array(y_val)
print(y_train)
print(x_train)

In [ ]:
print(x_train[5])

In [ ]:
print(y_train[5])